In [ ]:
%%flink_gateway_sql_prepare
DROP TABLE IF EXISTS `filter`;
CREATE TABLE `filter` (
`id` STRING,
`type` STRING,
`https://industry-fusion.com/types/v0.9/state` STRING,
`https://industry-fusion.com/types/v0.9/strength` STRING,
`https://industry-fusion.com/types/v0.9/hasCartridge` STRING,
`ts` TIMESTAMP(3) METADATA FROM 'timestamp') WITH (
  'connector' = 'kafka',
  'topic' = 'iff.ngsild.entities.filter',
  'properties.bootstrap.servers' = 'my-cluster-kafka-bootstrap:9092',
  'scan.startup.mode' = 'latest-offset',
  'format' = 'json'
);

DROP VIEW IF EXISTS `filter_view`;
CREATE VIEW `filter_view` AS
SELECT `id`,`type`,
`https://industry-fusion.com/types/v0.9/state`,
`https://industry-fusion.com/types/v0.9/strength`,
`https://industry-fusion.com/types/v0.9/hasCartridge`,
`ts` FROM (
  SELECT *,
ROW_NUMBER() OVER (PARTITION BY `id`
ORDER BY ts DESC) AS rownum
FROM `filter` )
WHERE rownum = 1;


In [ ]:
%%flink_gateway_sql_prepare
DROP TABLE IF EXISTS `cutter`;
CREATE TABLE `cutter` (
`id` STRING,
`type` STRING,
`https://industry-fusion.com/types/v0.9/state` STRING,
`https://industry-fusion.com/types/v0.9/hasFilter` STRING,
`https://industry-fusion.com/types/v0.9/hasInWorkpiece` STRING,
`https://industry-fusion.com/types/v0.9/hasOutWorkpiece` STRING,
`https://industry-fusion.com/oee/v0.9/availabilityState` STRING,

`ts` TIMESTAMP(3) METADATA FROM 'timestamp') WITH (
  'connector' = 'kafka',
  'topic' = 'iff.ngsild.entities.cutter',
  'properties.bootstrap.servers' = 'my-cluster-kafka-bootstrap:9092',
  'scan.startup.mode' = 'latest-offset',
  'format' = 'json'
);
DROP VIEW IF EXISTS `cutter_view`;
CREATE VIEW `cutter_view` AS
SELECT `id`,`type`,
`https://industry-fusion.com/types/v0.9/state`,
`https://industry-fusion.com/types/v0.9/hasFilter`,
`https://industry-fusion.com/types/v0.9/hasInWorkpiece`,
`https://industry-fusion.com/types/v0.9/hasOutWorkpiece`,
`https://industry-fusion.com/oee/v0.9/availabilityState`,
`ts` FROM (
  SELECT *,
ROW_NUMBER() OVER (PARTITION BY `id`
ORDER BY ts DESC) AS rownum
FROM `cutter` )
WHERE rownum = 1;


In [ ]:
%%flink_gateway_sql_prepare
DROP TABLE IF EXISTS `rdf`;
CREATE TABLE `rdf` (
`subject` STRING,
`predicate` STRING,
`object` STRING,
`index` INTEGER,
PRIMARY KEY(`subject`,`predicate`,`index`)  NOT ENFORCED
) WITH (
  'connector' = 'upsert-kafka',
  'topic' = 'iff.rdf',
  'properties.bootstrap.servers' = 'my-cluster-kafka-bootstrap:9092',
  'value.format' = 'json',
  'key.format' = 'json'
);
